## Deep Learning Prototype

## Imports

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Load Training CSV File

Displays dataframe for verification purposes.

In [2]:
# Load the training CSV file
csv_file = '../image_labels/bounding_box_label_test.csv'
training_df = pd.read_csv(csv_file)

image_directory = '../media/daytime_full_cockpit_shift_perspective/'
training_df['image_path'] = training_df['image_path'].apply(lambda x: os.path.join(image_directory, x))

training_df


,image_path,x,y,w,h
0,../media/daytime_full_cockpit_shift_perspectiv...,1060.0,650.0,130.0,120.0
1,../media/daytime_full_cockpit_shift_perspectiv...,1060.0,650.0,130.0,120.0
2,../media/daytime_full_cockpit_shift_perspectiv...,1060.0,650.0,130.0,120.0
3,../media/daytime_full_cockpit_shift_perspectiv...,1060.0,650.0,130.0,120.0
4,../media/daytime_full_cockpit_shift_perspectiv...,1060.0,650.0,130.0,120.0
...,...,...,...,...,...
3170,../media/daytime_full_cockpit_shift_perspectiv...,860.0,549.0,95.0,91.0
3171,../media/daytime_full_cockpit_shift_perspectiv...,860.0,549.0,95.0,91.0
3172,../media/daytime_full_cockpit_shift_perspectiv...,860.0,549.0,95.0,91.0
3173,../media/daytime_full_cockpit_shift_perspectiv...,860.0,549.0,95.0,91.0


## Helper Function - Load Image

In [3]:
def load_image(file_path):
    image = cv2.imread(file_path)

    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

## Load Images and apply labels from CSV

In [4]:
def load_and_resize_image(file_path, scale_factor=0.5):
    img = cv2.imread(file_path)
    
    # Get the original image dimensions
    original_height, original_width = img.shape[:2]
    
    # Resize the image based on the scale factor
    new_height = int(original_height * scale_factor)
    new_width = int(original_width * scale_factor)
    
    img = cv2.resize(img, (new_width, new_height))
    
    return img

In [5]:
image_path = '../media/daytime_full_cockpit_shift_perspective/daytime_full_cockpit_shift_perspective_0000.jpg'
resized_image = load_and_resize_image(image_path, scale_factor=0.5)

print(f"Resized Image - Width: {resized_image.shape[1]}, Height: {resized_image.shape[0]}")

Resized Image - Width: 960, Height: 540


In [6]:
scale_factor = 0.5

# Load images into a NumPy array
images = np.array([load_and_resize_image(file_path, scale_factor) for file_path in training_df['image_path']])

# Extract bounding box coordinates
bounding_boxes = training_df[['x', 'y', 'w', 'h']].values

resized_bounding_boxes = bounding_boxes * scale_factor

# Split the dataset into training and validation sets
X_train, X_val, bb_train, bb_val = train_test_split(
    images, bounding_boxes, test_size=0.2, random_state=42
)

KeyboardInterrupt: 

## Epoch Notes



## Execute - Run Training Model

In [8]:
# Define the model architecture for bounding box regression
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(540, 960, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='linear')  # 4 for bounding box coordinates (x, y, w, h)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Train the model
model.fit(X_train, bb_train, epochs=5, validation_data=(X_val, bb_val))

# Save the trained model
model.save('instrument_identifier.h5')

Epoch 1/5
80/80 [==============================] - 302s 4s/step - loss: 91259640.0000 - accuracy: 0.5067 - val_loss: 4832.7983 - val_accuracy: 0.6898
Epoch 2/5
80/80 [==============================] - 305s 4s/step - loss: 2638.0549 - accuracy: 0.7083 - val_loss: 2086.5127 - val_accuracy: 0.6929
Epoch 3/5
80/80 [==============================] - 295s 4s/step - loss: 1343.5165 - accuracy: 0.7346 - val_loss: 1338.6376 - val_accuracy: 0.7701
Epoch 4/5
80/80 [==============================] - 275s 3s/step - loss: 818.1143 - accuracy: 0.9008 - val_loss: 994.0252 - val_accuracy: 0.9953
Epoch 5/5
80/80 [==============================] - 255s 3s/step - loss: 573.7141 - accuracy: 0.9980 - val_loss: 823.2964 - val_accuracy: 0.9953


/opt/homebrew/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
